In [158]:
#run pip install yfinance pandas --user before running this file 
from yahoo_fin.stock_info import get_live_price
from yahoo_fin import options as op
import pandas as pd
from datetime import datetime
import math
import random

In [159]:
#get contract and underlying information 
contractInfo = input("Enter the contract symbol for your option: ")
numContracts = input('Enter the number of contracts: ')
#AAPL240719C00105000 <- example contract that you can use as input
tickerSymbol = contractInfo[0:4]

underlyingPrice = get_live_price(tickerSymbol)

year = int(contractInfo[4:6])
month = int(contractInfo[6:8])
day = int(contractInfo[8:10])
expDate = datetime(2000 + year, month, day)

callOrPut = contractInfo[10]
strike = int(contractInfo[11:]) / 1000 

#get IV for option
def getIV():
  chain = []
  if callOrPut == 'C':
    chain = op.get_calls(tickerSymbol)
  else:
    chain = op.get_calls(tickerSymbol)
  
  chain = pd.DataFrame(chain)
  
  mask = chain['Strike'] == strike
  volPercent = chain[mask].head().to_numpy()[0][10]
  return volPercent[0:len(volPercent) - 1]

In [160]:
def getBlackScholesDelta():
  S = float(underlyingPrice) #underlying price
  K = float(strike) #strike
  r = 0.05 #risk free return 
  T = float((expDate - datetime.today()).days) #days to maturity
  vol = float(getIV()) #underlying vol
  
  #https://www.simtrade.fr/blog_simtrade/option-greeks-delta/
  delta = (math.log(S/K) + T * (r + math.pow(vol, 2) / 2)) / (vol * math.sqrt(T))
  
  if callOrPut == 'P':
    return 1 - delta
  return delta

In [161]:
history = [] #all purchases (adjustments) to the initial options purchase

def adjust(delta):
  if delta > 0:
    print(f"Sell {delta} shares of underlying")
    buySellUnderlying(-1 * delta)
  elif delta < 0:
    print(f"Buy {delta * -1} shares of underlying")
    buySellUnderlying(-1 * delta)
  else:
    print("No adjustments are required")

def buySellUnderlying(amount):
  history.append([amount, get_live_price(tickerSymbol)])
  #TODO: SEND AN EMAIL HERE SAYING HOW MUCH YOU WILL ADJUST BY
  
def getPositionDelta():
  #get delta of shares
  numShares = 0
  for purchase in history:
    numShares += purchase[1]
  
  #add shares delta to options delta
  return getBlackScholesDelta() + numShares

In [162]:
#initial spread
optionDelta = getBlackScholesDelta()
print(f"Your initial spread is:")
adjust(optionDelta)

Your initial spread is:
Sell 1093.4314272661052 shares of underlying


In [163]:
daysTillExpiration = (expDate - datetime.today()).days
threshold = input("Enter the threshold for the delta as decimal between 0.1 and 1(determines how frequently you will adjust): ")

#assuming one adjustment per day
for i in range(daysTillExpiration):   
  underlyingPrice += random.randint(-3, 3) #simulate new prices
  
  #check if delta has deviated from 0 by the threshold
  positionDelta = getPositionDelta()
  
  print(f"new delta: {positionDelta}")
  
  if abs(positionDelta) > threshold:
    adjust(positionDelta)

new delta: 1264.161430958963
Sell 1264.161430958963 shares of underlying
new delta: 1434.8914240467823
Sell 1434.8914240467823 shares of underlying
new delta: 1605.62142503861
Sell 1605.62142503861 shares of underlying
new delta: 1776.35141814158
Sell 1776.35141814158 shares of underlying
new delta: 1947.0814112293995
Sell 1947.0814112293995 shares of underlying


KeyboardInterrupt: 